In [1]:
Install-Module -Name Az.ApplicationInsights -RequiredVersion 1.0.3 -Scope CurrentUser 

The below script needs to be able to find the current output cell; this is an easy method to get it.


Untrusted repository
You are installing the modules from an untrusted repository. If you trust this repository, change its InstallationPolicy value by running the Set-PSRepository cmdlet. Are you sure you want to install the modules from 'PSGallery'?
[Y] Yes  [A] Yes to All  [N] No  [L] No to All  [S] Suspend  [?] Help(default is 'N')Select: a


Install-Package: C:\Users\finke\.dotnet\tools\.store\microsoft.dotnet-interactive\1.0.120301\microsoft.dotnet-interactive\1.0.120301\tools\netcoreapp3.1\any\Modules\PowerShellGet\PSModule.psm1:9685
Line |
9685 |  … talledPackages = PackageManagement\Install-Package @PSBoundParameters
     |                     ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | The version '1.0.3' of the module 'Az.ApplicationInsights' being installed is not catalog
     | signed. Ensure that the version '1.0.3' of the module 'Az.ApplicationInsights' has the catalog
     | file 'Az.ApplicationInsights.cat' and signed with the same publisher 'CN=Microsoft
     | Corporation, O=Microsoft Corporation, L=Redmond, S=Washington, C=US' as the
     | previously-installed module 'Az.ApplicationInsights' with version '1.1.0' under the directory
     | 'C:\Users\finke\OneDrive\Documents\WindowsPowerShell\Modules\Az.ApplicationInsights\1.1.0'. If
     | you still want to install or update, use -SkipPublish

In [2]:
Get-PackageProvider


Name                     Version          DynamicOptions
----                     -------          --------------
NuGet                    3.0.0.1          Destination, ExcludeVersion, Scope, SkipDependencies, He…
PowerShellGet            2.2.3.0          PackageManagementProvider, Type, Scope, AllowClobber, Sk…



In [3]:
Connect-AzAccount


Account            SubscriptionName TenantId                             Environment
-------            ---------------- --------                             -----------
finked@hotmail.com Start-Automating 58475834-3702-4a8b-966f-39caa3ebad39 AzureCloud



In [5]:
Get-AzContext


Name                                     Account        SubscriptionNa Environment    TenantId
                                                        me
----                                     -------        -------------- -----------    --------
Start-Automating (9b5a0e7f-50d3-4f01-83… finked@hotmai… Start-Automat… AzureCloud     58475834-370…



In [6]:
Set-AzContext -Subscription "00a9decb-e45c-4ef1-839b-799c5d0b21f8"


Name                                     Account        SubscriptionNa Environment    TenantId
                                                        me
----                                     -------        -------------- -----------    --------
Windows Azure MSDN - Visual Studio Ulti… finked@hotmai… Windows Azure… AzureCloud     3b8a5a65-6dc…



In [7]:
Install-Module -Name Az.Monitor -Scope CurrentUser


Untrusted repository
You are installing the modules from an untrusted repository. If you trust this repository, change its InstallationPolicy value by running the Set-PSRepository cmdlet. Are you sure you want to install the modules from 'PSGallery'?
[Y] Yes  [A] Yes to All  [N] No  [L] No to All  [S] Suspend  [?] Help(default is 'N')Select: a


In [9]:
Import-Module Az.Monitor
$logs = Get-AzLog -MaxRecord 200 | Select-Object -ExcludeProperty Authorization,Claims,EventTimestamp,HttpRequest,Level,Properties,SubmissionTimestamp
$logJson = ConvertTo-Json $logs -Depth 3
$logJson | Out-Display -MimeType "application/json"

Write-Error: This module requires Az.Accounts version 1.8.0. An earlier version of Az.Accounts is imported in the current PowerShell session. Please open a new session before importing this module. This error could indicate that multiple incompatible versions of the Azure PowerShell cmdlets are installed on your system. Please see https://aka.ms/azps-version-error for troubleshooting information.
Import-Module: The module to process 'Az.Monitor.psm1', listed in field 'ModuleToProcess/RootModule' of module manifest 'C:\Users\finke\OneDrive\Documents\WindowsPowerShell\Modules\Az.Monitor\2.0.0\Az.Monitor.psd1' was not processed because no valid module was found in any module directory.
Get-AzLog: 
Line |
   2 |  $logs = Get-AzLog -MaxRecord 200 | Select-Object -ExcludeProperty Aut …
     |          ~~~~~~~~~
     | The 'Get-AzLog' command was found in the module 'Az.Monitor', but the module could not be loaded. For more information, run 'Import-Module Az.Monitor'.


In [ ]:
$entryByResourceGroup = [Graph.Histogram]::new()
$entryByResourceGroup.name = "By ResourceGroup"
$entryByResourceGroup.x =  $logs.ResourceGroupName

$entryByResourceProvider = [Graph.Histogram]::new()
$entryByResourceProvider.name = "By ResourceProvider"
$entryByResourceProvider.x =  $logs.ResourceProviderName.value

$entryByStatus = [Graph.Histogram]::new()
$entryByStatus.name = "By Status"
$entryByStatus.x =  $logs.Status.value


New-PlotlyChart -Trace @($entryByResourceGroup,$entryByResourceProvider,$entryByStatus) -Title "Events" | Out-Display

In [ ]:
$entryByResourceProviderSuccess = [Graph.Histogram]::new()
$entryByResourceProviderSuccess.name = "Success By ResourceProvider"
$entryByResourceProviderSuccess.x = ($logs | where-object { $_.Status.value -eq "Succeeded"}).ResourceProviderName.value

$entryByResourceProviderFailure = [Graph.Histogram]::new()
$entryByResourceProviderFailure.name = "Failure By ResourceProvider"
$entryByResourceProviderFailure.x =  ($logs | where-object { $_.Status.value -ne "Succeeded"}).ResourceProviderName.value

$layout = [Layout]::new()
$layout.barmode = "stack"
New-PlotlyChart -Layout $layout -Trace @($entryByResourceProviderSuccess, $entryByResourceProviderFailure) -Title "Events outcome by Resource Provider" | Out-Display